In [ ]:
import pandas as pd
import numpy as np
from model import *
import math

In [ ]:
data_path = "../final_data/"
category_data = pd.read_csv(data_path + "all_category_ec.csv")
neg_rate = pd.read_csv(data_path + "all_-99_rate.csv")
public_upc = pd.read_csv(data_path + "pd_pos_all1718_public.csv")
upc = pd.read_csv(data_path + "1718upc_cleaned.csv")
ec = pd.read_csv(data_path + "1718ec_cleaned.csv")

In [ ]:
# just load 1718 ppc data for data check, will remove it later
ppc = pd.read_csv(data_path + "1718ppc_cleaned.csv")

In [ ]:
# Jason: This is EC code, which is public data. The negative numbers in the rest of the notebook are the same.
category_data = category_data.drop(["-90.0","-70.0"],axis = 1)

In [ ]:
#combine category information from original upc data source
upc=upc.merge(public_upc,left_on="upc_code",right_on="upc",how = "left").drop_duplicates(subset=["upc_code"])[["upc_code","upc_description","category"]]
upc.index = range(upc.shape[0])

In [ ]:
#list context category types
ca_list = category_data["category"].tolist()
#list context ec types
ca_ec = category_data.columns.tolist()[1:]
ca_ec = [int(float(i)) for i in ca_ec]
#lsit ec types in data we want to match 
ec_list = ec.ec_code.tolist()

In [ ]:
def out_list(a,b):
    return list(set(a)-set(b))
def intersect(a,b):
    return list(set(a)&set(b))

In [ ]:
out_ec = out_list(ec_list,ca_ec)

In [ ]:
def get_pot_ec(upc,ec):
    cat = upc.iloc[0,2]
    if cat!=cat: #check missing category
        return -80
    elif cat not in ca_list:
        return ec.ec_code.tolist()
    else:
        x=category_data[category_data.category==cat]
        cat_ec = [i for i in x.columns if (x[i]==1).all()]
        all_ec = cat_ec + out_ec
        if len(cat_ec)<=5:
            return cat_ec
        elif "-99.0" not in cat_ec:
                return all_ec
        else:
            rate = neg_rate[neg_rate["category"]==cat].iloc[0,3]
            if rate>=0.1:
                return ["-99.0"]+all_ec
            else:
                return all_ec
            

In [ ]:
upc = pd.read_csv('../final_data/subsets/subset1.csv')

In [ ]:

data_to_save = pd.DataFrame(columns = ['upc', 'ec', 'confidence'])
for i in tqdm(range(upc.shape[0])):
    #%run model.py
    c = get_pot_ec(upc.iloc[[i]],ec)
    if c==-80:
        # We are quite sure these upc's ec is -80
        output = pd.DataFrame([[upc.iloc[i,0],-80,1]],columns = ['upc', 'ec', 'confidence'])
    elif len(c)<=5:
        ec_data = ec[ec.ec_code.isin([int(float(j)) for j in c])]
        if ec_data.shape[0]==0:
            output = pd.DataFrame(columns = ['upc', 'ec', 'confidence'])
        else:
            output = test_other_data(upc.iloc[[i]],ec_data,0.738,0.262)  #0.738,0.262
        if "-99.0" in c:
            if neg_rate[neg_rate.category==upc.iloc[i,2]].shape[0]!=0:
                output = pd.concat([output,pd.DataFrame([[upc.iloc[i,0],-99,float(neg_rate[neg_rate.category==upc.iloc[i,2]]["-99_rate"])]],columns = ['upc', 'ec', 'confidence'])])
            else:
                output = pd.concat([output,pd.DataFrame([[upc.iloc[i,0],-99,1]],columns = ['upc', 'ec', 'confidence'])])
    else:
        if len([i for i in c if i=="-99.0"])==2:
            ec_data = ec[ec.ec_code.isin([int(float(j)) for j in c])]
            output = test_other_data(upc.iloc[[i]],ec_data,0.738,0.262).iloc[:4,:]
            output = pd.concat([output,pd.DataFrame([[upc.iloc[i,0],-99,float(neg_rate[neg_rate.category==upc.iloc[i,2]]["-99_rate"])]],columns = ['upc', 'ec', 'confidence'])])
        else:
            ec_data = ec[ec.ec_code.isin([int(float(j)) for j in c])]
            output = test_other_data(upc.iloc[[i]],ec_data,0.738,0.262)
    if output.shape[0]<5:
        output2 = test_other_data(upc.iloc[[i]],ec[~ec.ec_code.isin(output.ec)],0.738,0.262).iloc[:(5-output.shape[0]),:]
        output = pd.concat([output,output2])
    output = output.sort_values(["confidence"],ascending=False)
    data_to_save = pd.concat([data_to_save, output])
    data_to_save.index = range(data_to_save.shape[0]) 
    
    
    
    
    #save data each step(we do this because workspace may auto close)
    data_to_save.to_csv("../final_data/subset_result/submission1.csv",index = False)
    ground_truth = ppc[ppc.upc.isin(data_to_save.upc.unique())]
    ground_truth.to_csv("../final_data/subset_result/ground_truth1.csv",index = False)
print("finish")